In [10]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import geopandas

Ya tenemos lo que queremos priorizar, lo cual es, en orden de mas prioritario a menos
- Big starup
- Vegan
- Starbucks
- Vet
- Bar

In [11]:
load_dotenv()

True

In [12]:
client_id = os.getenv('Client_Id')
client_secret = os.getenv('Client_Secret')


In [13]:
client_secret

'5DYNBHMUAHF12PPNJMU1ACHI4AXFZZZRBNSDBW5Y4Y0VXVJL'

In [14]:
url_query = 'https://api.foursquare.com/v2/venues/search'

In [15]:
def getFromDict(diccionario, mapa):
    '''
    Esta función bucea por el diccionario que le damos en base a un mapa (ruta) que también se le da.
    Dentro de el diccionario irá a donde el mapa le indique.
    Recibe : diccionario y mapa ('hoja de ruta')
    Devuelve: lo espcificado dentro del diccionario pr el mapa
    '''
    from functools import reduce
    import operator
    return reduce(operator.getitem,mapa,diccionario)

In [93]:
def getAPI(url_query, coordenadas, query): #coordenadas en string, latitud y longitud (en ese orden)
    '''
    Esta función llama a la API en base a unas coordenadas y a una query que le demos (restaurantes, gimansios, etc.) 
    y nos devuelve un json.
    Recibe : url de la API, coordenadas (en formato string), query (qué queremos que nos busque)
    Devuelve: json en base a los parámetros establecidos
    '''
    parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f'{coordenadas}',
    "query": f"{query}",
    "limit": 200   
}
    resp = requests.get(url_query, params=parametros).json()['response']['venues']
    
    return resp

In [92]:
def limpiezaresp(json):
    '''
    Esta función limpia la respuesta json que nos devuelve la llamada a la API.
    Dentro del for loop llama a la función getFromDict con los mapas(rutas), establecidos en esta función,
    y coge lo que le indicamos dentro de esa ruta y lo coloca ordenado según lo que le indiquemos (nombre, logitud, etc).
    Estas itercaiones las coloca en un json nuevo (jason), al que se va apendando cada iteración. Para meter en el dataframe 
    final que queremos la columna de tipo, generamos el for dentro del for para llegar a la indicación 'name' y que nos 
    meta en el df una nueva columan 'tipo'
    
    Recibe: json
    Devuelve: dataframe con 6 columnas
    
    '''
    mapa_nombre = ['name']
    mapa_latitud = ['location','lat']
    mapa_longitud = ['location', 'lng']
    mapa_ciudad = ['location','city']
    mapa_distancia = ['location','distance']
    
    
    jason = []
    
    for dic in json:
        try:
            diccio= {}
            diccio['nombre']= getFromDict(dic,mapa_nombre)
            diccio['latitud']= getFromDict(dic,mapa_latitud)
            diccio['longitud']= getFromDict(dic,mapa_longitud)
            diccio['ciudad']= getFromDict(dic,mapa_ciudad)
            diccio['distancia'] = getFromDict(dic,mapa_distancia)
            jason.append(diccio)
        except:
            pass
    
    df = pd.DataFrame(jason)
    
    tipo = []
    for i in json:
        for z in i["categories"]:
            tipo.append(z["name"])
    tipo = pd.Series(tipo)
    
    df['tipo'] = tipo
    
    return df

In [89]:
def todo(url_query, coordenadas, query):
    '''
    Esta función resume la función de la llamada a la API y la limpieza del json, llama a las dos.
    Recibe: la url de la API, las coordenadas de la localización a estudiar y la query que queremos que nos devuelva la API
    Devuelve: el json limpio con la información justa que queremos.
    '''
    api = getAPI(url_query, coordenadas, query)
    return limpiezaresp(api)

In [90]:
lista = ['Vegan', 'Start Up', 'Veterinary', 'Bar', 'Starbucks']

In [87]:
df_full_ny = []
for i in lista:
    df_ny = todo('https://api.foursquare.com/v2/venues/search','40.75673, -73.98996', i)
    df_full_ny.append(df_ny)
df_NewYork = pd.concat(df_full_ny)
df_NewYork.head()

,nombre,latitud,longitud,ciudad,distancia,tipo
0,Omv! Vegan Burgers & More,40.757819,-73.985537,New York,392,Vegetarian / Vegan Restaurant
1,Marty's Vegan Fast Foods,40.750163,-73.988871,New York,736,Food Truck
2,MTHR Vegan,40.762373,-73.987867,New York,652,Vegetarian / Vegan Restaurant
3,Happy Zoe Vegan Bakery,40.719246,-73.944252,New York,5681,Bakery
4,Maoz Vegan,40.751706,-73.974375,New York,1428,Vegetarian / Vegan Restaurant


In [14]:
#Vamos a hacer lo mismo para Madrid 

In [75]:
df_full_mad = []
for i in lista:
    df_mad = todo('https://api.foursquare.com/v2/venues/search','40.50222, -3.67097', i)
    df_full_mad.append(df_mad)
df_Mad = pd.concat(df_full_mad)
df_Mad.head()

,nombre,latitud,longitud,ciudad,distancia,tipo
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,Madrid,6696,Vegetarian / Vegan Restaurant
1,Vegan Streets Madrid Centro,40.417512,-3.697636,Madrid,9696,Vegetarian / Vegan Restaurant
2,Vegan bistro,40.419410,-3.679066,Madrid,9243,Burger Joint
3,vegetarian / vegan restaurant,40.432974,-3.686693,Madrid,7822,Vegetarian / Vegan Restaurant
4,THUNDER VEGAN FOOD,40.424450,-3.702513,Madrid,9060,Vegetarian / Vegan Restaurant


In [16]:
#Ahora lo mismo para Barcelona

In [76]:
df_full_bcn = []
for i in lista:
    df_bcn = todo('https://api.foursquare.com/v2/venues/search','41.40080, 2.19763', i)
    df_full_bcn.append(df_bcn)
df_Bcn = pd.concat(df_full_bcn)
df_Bcn.head()

,nombre,latitud,longitud,ciudad,distancia,tipo
0,Vegan Streets Barcelona Glories,41.410040,2.193974,Barcelona,1072.0,Vegetarian / Vegan Restaurant
1,Vegan Junk Food Bar,41.383386,2.180255,Barcelona,2421.0,Vegetarian / Vegan Restaurant
2,Monchito Vegan,41.380971,2.179880,Barcelona,2658.0,Vegetarian / Vegan Restaurant
3,ENJOY VEGAN,41.388152,2.180673,Barcelona,1996.0,Vegetarian / Vegan Restaurant
4,"Good Shit, Vegan Kebabs",41.402854,2.156511,Barcelona,3441.0,Shoe Store


Ya tenemos los tres dataframes que queremos.
Cada uno nos muestra todas nuestras 3 variables que tenemos en cuenta en esa ciudad, es decir, df_NewYork, me muestra todos los Veganos, todos los Starbucks, todos los Veterinarios, todos los Bares y todas las Start Ups.

DataFrames chulos:
- df_NewYork
- df_Mad
- df_Bcn

Ahora lo que tenemos que hacer es convertirlos en geodataframes y meterles una nueva columna que sean las coordenadas en tipo point para poder hacer las geoqueries de la manera correspondiente

In [18]:
# mira que guapa
# pero no la puedo usar, porque lo que viene despues, que son cosas de terminal que solo funcionan con mac...
'''
def export_to_Json (name, df):
    with open(f'{name}', 'w') as f:
        json.dump(df, f)
'''

"\ndef export_to_Json (name, df):\n    with open(f'{name}', 'w') as f:\n        json.dump(df, f)\n"

Nos exportamos estos 3 dataframes maravillosos en formato .csv para cargarlos en Mongo

In [91]:
df_NewYork.to_csv('newyork_data.csv')
df_Mad.to_csv('mad_data.csv')
df_Bcn.to_csv('bcn_data.csv')